In [1]:
import pandas as pd

In [2]:
import perprocess

In [3]:
health_cost = pd.read_excel("data/OR2021_SSS_ARPA_Partial.xlsx",sheet_name = "By Family")

In [4]:
health_cost.head()

,Status,Family Type,Adult(s),Infant(s),Preshooler(s),Schoolager(s),Teenager(s),State,Year,All Families Table #,...,State Income Taxes,Federal Child Tax Credit (-),Hourly Self-Sufficiency Wage,Monthly Self-Sufficiency Wage,Annual Self-Sufficiency Wage,Emergency Savings,Federal & Oregon EITC,Federal CDCTC,Oregon WFHDC,Total Annual Resources
0,Partial,a1i0p0s0t0,1,0,0.0,0.0,0.0,OR,2021,1,...,99.27,0.0,10.55,1856.08,22272.93,28.28,0.00,0.00,0.00,22272.93
1,Partial,a1i1p0s0t0,1,1,0.0,0.0,0.0,OR,2021,1,...,196.15,-300.0,20.88,3675.60,44107.22,76.31,0.00,4000.00,943.64,49050.86
2,Partial,a1i0p1s0t0,1,0,1.0,0.0,0.0,OR,2021,1,...,182.70,-300.0,19.89,3501.02,42012.22,75.18,25.88,3983.23,1433.96,47455.30
3,Partial,a1i0p0s1t0,1,0,0.0,1.0,0.0,OR,2021,1,...,182.37,-250.0,19.87,3496.63,41959.60,70.77,35.05,3113.07,933.92,46041.64
4,Partial,a1i0p0s0t1,1,0,0.0,0.0,1.0,OR,2021,1,...,125.84,-250.0,15.70,2762.47,33149.61,66.02,1569.59,0.00,0.00,34719.20


In [5]:
health_cost.columns

Index(['Status', 'Family Type', 'Adult(s)', 'Infant(s)', 'Preshooler(s)',
       'Schoolager(s)', 'Teenager(s)', 'State ', 'Year',
       'All Families Table #', 'County', 'Housing Costs', 'Child Care Costs',
       'Food Costs ', 'Transportation Costs', 'Health Care (Net) Costs ',
       'Premium', 'Out-of-Pocket', 'Miscellaneous costs', 'Taxes (Net)',
       'Federal Income Taxes', 'Payroll Taxes', 'State Sales Taxes',
       'State Income Taxes ', 'Federal Child Tax Credit (-)',
       'Hourly Self-Sufficiency Wage                           ',
       'Monthly Self-Sufficiency Wage                           ',
       'Annual Self-Sufficiency Wage                           ',
       'Emergency Savings', 'Federal & Oregon EITC', 'Federal CDCTC',
       'Oregon WFHDC ', 'Total Annual Resources'],
      dtype='object')

In [6]:
health_cost_clean_col = perprocess.std_col_names(health_cost)

In [7]:
health_cost_clean_col.columns

Index(['analysis', 'family_type', 'adult', 'infant', 'preschooler',
       'schoolager', 'teenager', 'state', 'year', 'all_families_table_#',
       'place', 'housing', 'child_care', 'food', 'transportation',
       'health_care', 'premium', 'out_of_pocket', 'miscellaneous', 'taxes',
       'federal_income_taxes', 'payroll_taxes', 'state_sales_taxes',
       'state_income_taxes', 'federal_child_tax_credit',
       'hourly_self_sufficiency_wage', 'monthly_self_sufficiency_wage',
       'annual_self_sufficiency_wage', 'emergency_savings',
       'federal_and_oregon_eitc', 'federal_cdctc', 'oregon_wfhdc',
       'total_annual_resources'],
      dtype='object')

In [8]:
health_cost_clean_col.dtypes

analysis                          object
family_type                       object
adult                              int64
infant                             int64
preschooler                      float64
schoolager                       float64
teenager                         float64
state                             object
year                               int64
all_families_table_#               int64
place                             object
housing                          float64
child_care                       float64
food                             float64
transportation                   float64
health_care                      float64
premium                          float64
out_of_pocket                    float64
miscellaneous                    float64
taxes                            float64
federal_income_taxes             float64
payroll_taxes                    float64
state_sales_taxes                  int64
state_income_taxes               float64
federal_child_ta

In [9]:
from sqlalchemy import (create_engine, MetaData, 
                        Table, Column, 
                        Integer, String, 
                        Float, ForeignKey)
from sqlalchemy.orm import declarative_base

In [10]:
engine = create_engine("sqlite:///sss.sqlite", echo = False)
m = MetaData(bind=engine)
Base = declarative_base(metadata=m)

In [11]:
class HealthCare(Base):
    __tablename__ = "health_care"
    family_type = Column("family_type", String, primary_key = True)
    state = Column("state", String, primary_key = True)
    place = Column("place", String, primary_key = True)
    year = Column("year", Integer, primary_key = True)
    analysis =  Column("analysis", String, primary_key = True)
    premium = Column("premium", Float)
    out_of_pocket = Column("out_of_pocket", Float)
Base.metadata.create_all(engine)

In [12]:
from sqlalchemy.orm.session import sessionmaker

In [13]:
Session = sessionmaker(bind=engine)
session = Session()

In [14]:
health_cost_clean_col.to_dict(orient="records")

[{'analysis': 'Partial',
  'family_type': 'a1i0p0s0t0',
  'adult': 1,
  'infant': 0,
  'preschooler': 0.0,
  'schoolager': 0.0,
  'teenager': 0.0,
  'state': 'OR',
  'year': 2021,
  'all_families_table_#': 1,
  'place': 'Baker County',
  'housing': 637.0,
  'child_care': 0.0,
  'food': 269.37,
  'transportation': 289.19,
  'health_care': 197.11,
  'premium': 124.37,
  'out_of_pocket': 72.74,
  'miscellaneous': 139.27,
  'taxes': 324.14,
  'federal_income_taxes': 81.02,
  'payroll_taxes': 143.85,
  'state_sales_taxes': 0,
  'state_income_taxes': 99.27,
  'federal_child_tax_credit': 0.0,
  'hourly_self_sufficiency_wage': 10.55,
  'monthly_self_sufficiency_wage': 1856.08,
  'annual_self_sufficiency_wage': 22272.93,
  'emergency_savings': 28.28,
  'federal_and_oregon_eitc': 0.0,
  'federal_cdctc': 0.0,
  'oregon_wfhdc': 0.0,
  'total_annual_resources': 22272.93},
 {'analysis': 'Partial',
  'family_type': 'a1i1p0s0t0',
  'adult': 1,
  'infant': 1,
  'preschooler': 0.0,
  'schoolager': 0.0,


In [15]:
session.bulk_insert_mappings(HealthCare,health_cost_clean_col.to_dict(orient="records"))

In [16]:
session.commit()

In [17]:
# class GeoIdentifier(Base):
#     __tablename__ = 'geo_identifier'
#     state = Column('state', String, primary_key = True)
#     place = Column('place', String, primary_key = True)
#     state_fips = Column('state_fips', String, primary_key = True)
#     county_fips = Column('county_fips', String, primary_key = True)
#     place_fips =  Column('place_fips', String, primary_key = True)
#     #census_year = Column('census_year', Integer)
#     cpi_region = Column('cpi_region', String)
# Base.metadata.create_all(engine)

In [18]:
right = pd.read_excel("StateAbbreviation_Regions_07192022_AKu.xlsx")
right.head()

,State Name,USPS Abbreviation,CPI Region
0,Alabama,AL,South
1,Alaska,AK,West
2,Arizona,AZ,West
3,Arkansas,AR,South
4,California,CA,West


In [19]:
left = pd.read_excel("SSScounty-place-list.xlsx", converters={"FIPS_2010_town":str})
left.head()

,FIPS,state_alpha,areaname,countyname,county_town_name,FIPS_2010_town
0,1001.0,AL,"Montgomery, AL MSA",Autauga County,Autauga County,0100199999
1,1003.0,AL,"Daphne-Fairhope-Foley, AL MSA",Baldwin County,Baldwin County,0100399999
2,1005.0,AL,"Barbour County, AL",Barbour County,Barbour County,0100599999
3,1007.0,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",Bibb County,Bibb County,0100799999
4,1009.0,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",Blount County,Blount County,0100999999


In [20]:
geo_merge = pd.merge(left, right,how='left', left_on="state_alpha",right_on="USPS Abbreviation")


In [21]:
geo_merge["state_fips"] = geo_merge["FIPS_2010_town"].str[:2]

In [22]:
geo_merge.head()

,FIPS,state_alpha,areaname,countyname,county_town_name,FIPS_2010_town,State Name,USPS Abbreviation,CPI Region,state_fips
0,1001.0,AL,"Montgomery, AL MSA",Autauga County,Autauga County,0100199999,Alabama,AL,South,01
1,1003.0,AL,"Daphne-Fairhope-Foley, AL MSA",Baldwin County,Baldwin County,0100399999,Alabama,AL,South,01
2,1005.0,AL,"Barbour County, AL",Barbour County,Barbour County,0100599999,Alabama,AL,South,01
3,1007.0,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",Bibb County,Bibb County,0100799999,Alabama,AL,South,01
4,1009.0,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",Blount County,Blount County,0100999999,Alabama,AL,South,01


In [23]:
geo_merge["county_fips"] = geo_merge["FIPS_2010_town"].str[2:5]

In [24]:
geo_merge.head()

,FIPS,state_alpha,areaname,countyname,county_town_name,FIPS_2010_town,State Name,USPS Abbreviation,CPI Region,state_fips,county_fips
0,1001.0,AL,"Montgomery, AL MSA",Autauga County,Autauga County,0100199999,Alabama,AL,South,01,001
1,1003.0,AL,"Daphne-Fairhope-Foley, AL MSA",Baldwin County,Baldwin County,0100399999,Alabama,AL,South,01,003
2,1005.0,AL,"Barbour County, AL",Barbour County,Barbour County,0100599999,Alabama,AL,South,01,005
3,1007.0,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",Bibb County,Bibb County,0100799999,Alabama,AL,South,01,007
4,1009.0,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",Blount County,Blount County,0100999999,Alabama,AL,South,01,009


In [25]:
geo_merge.columns

Index(['FIPS', 'state_alpha', 'areaname', 'countyname', 'county_town_name',
       'FIPS_2010_town', 'State Name', 'USPS Abbreviation', 'CPI Region',
       'state_fips', 'county_fips'],
      dtype='object')

In [26]:
geo_merge["place_fips"] = geo_merge["FIPS_2010_town"].str[5:]

In [27]:
geo_merge.head()

,FIPS,state_alpha,areaname,countyname,county_town_name,FIPS_2010_town,State Name,USPS Abbreviation,CPI Region,state_fips,county_fips,place_fips
0,1001.0,AL,"Montgomery, AL MSA",Autauga County,Autauga County,0100199999,Alabama,AL,South,01,001,99999
1,1003.0,AL,"Daphne-Fairhope-Foley, AL MSA",Baldwin County,Baldwin County,0100399999,Alabama,AL,South,01,003,99999
2,1005.0,AL,"Barbour County, AL",Barbour County,Barbour County,0100599999,Alabama,AL,South,01,005,99999
3,1007.0,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",Bibb County,Bibb County,0100799999,Alabama,AL,South,01,007,99999
4,1009.0,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",Blount County,Blount County,0100999999,Alabama,AL,South,01,009,99999


In [28]:
geo_merge.columns=["FIPS", "state", "areaname", "countyname", "place",
       "FIPS_2010_town", "State Name", "USPS Abbreviation", "cpi_region",
       "state_fips", "county_fips","place_fips"]

In [29]:
geo_merge.isna().sum()

FIPS                 0
state                0
areaname             0
countyname           0
place                0
FIPS_2010_town       3
State Name           1
USPS Abbreviation    1
cpi_region           1
state_fips           3
county_fips          3
place_fips           3
dtype: int64

In [30]:
geo_merge[geo_merge["state_fips"].isna()]

,FIPS,state,areaname,countyname,place,FIPS_2010_town,State Name,USPS Abbreviation,cpi_region,state_fips,county_fips,place_fips
1401,23003.0,ME,"Aroostook County, ME",Aroostook County,Oxbow plantation,NaN,Maine,ME,Northeast,NaN,NaN,NaN
1464,23007.0,ME,"Franklin County, ME",Franklin County,Madrid town,NaN,Maine,ME,Northeast,NaN,NaN,NaN
1801,23029.0,ME,"Washington County, ME",Washington County,Centerville town,NaN,Maine,ME,Northeast,NaN,NaN,NaN


In [31]:
geo_merge = geo_merge.dropna()

In [32]:
engine = create_engine("sqlite:///sss.sqlite", echo = False)
m = MetaData(bind=engine)
Base = declarative_base(metadata=m)

In [33]:
class GeoIdentifier(Base):
    __tablename__ = "geo_identifier"
    state = Column("state", String, primary_key = True)
    place = Column("place", String, primary_key = True)
    state_fips = Column("state_fips", String)
    county_fips = Column("county_fips", String)
    countyname = Column("countyname", String, primary_key = True)
    place_fips =  Column("place_fips", String, primary_key = True)
    #census_year = Column('census_year', Integer)
    cpi_region = Column("cpi_region", String)
Base.metadata.create_all(engine)

In [34]:
Session = sessionmaker(bind=engine)
session = Session()

In [35]:
session.bulk_insert_mappings(GeoIdentifier,geo_merge.to_dict(orient="records"))

In [36]:
geo_merge.to_dict(orient="records")

[{'FIPS': 1001.0,
  'state': 'AL',
  'areaname': 'Montgomery, AL MSA',
  'countyname': 'Autauga County',
  'place': 'Autauga County',
  'FIPS_2010_town': '0100199999',
  'State Name': 'Alabama',
  'USPS Abbreviation': 'AL',
  'cpi_region': 'South',
  'state_fips': '01',
  'county_fips': '001',
  'place_fips': '99999'},
 {'FIPS': 1003.0,
  'state': 'AL',
  'areaname': 'Daphne-Fairhope-Foley, AL MSA',
  'countyname': 'Baldwin County',
  'place': 'Baldwin County',
  'FIPS_2010_town': '0100399999',
  'State Name': 'Alabama',
  'USPS Abbreviation': 'AL',
  'cpi_region': 'South',
  'state_fips': '01',
  'county_fips': '003',
  'place_fips': '99999'},
 {'FIPS': 1005.0,
  'state': 'AL',
  'areaname': 'Barbour County, AL',
  'countyname': 'Barbour County',
  'place': 'Barbour County',
  'FIPS_2010_town': '0100599999',
  'State Name': 'Alabama',
  'USPS Abbreviation': 'AL',
  'cpi_region': 'South',
  'state_fips': '01',
  'county_fips': '005',
  'place_fips': '99999'},
 {'FIPS': 1007.0,
  'state

In [37]:
session.commit()

In [38]:
AR2022 = pd.read_excel("data/AR2022_SSS_Full.xlsx",sheet_name = "By Family")

In [39]:
AR2022_clean_col = perprocess.std_col_names(AR2022)

In [40]:
AR2022_clean_col.head()

,analysis,family_type,adult,infant,preschooler,schoolager,teenager,state,year,all_families_table_#,...,broadband_and_cell_phone,other_necessities,taxes,earned_income_tax_credit,child_care_tax_credit,child_tax_credit,hourly_self_sufficiency_wage,monthly_self_sufficiency_wage,annual_self_sufficiency_wage,emergency_savings
0,Full,a1i0p0s0t0,1,0,0.0,0.0,0.0,AR,2022,1,...,182.0,132.69,301.30,0.00,0.0,0.00,11.04,1942.84,23314.03,48.40
1,Full,a1i1p0s0t0,1,1,0.0,0.0,0.0,AR,2022,1,...,182.0,261.95,576.95,-39.08,-55.0,-166.67,19.20,3379.68,40556.16,101.52
2,Full,a1i0p1s0t0,1,0,1.0,0.0,0.0,AR,2022,1,...,182.0,258.78,562.07,-48.53,-55.0,-166.67,18.87,3320.51,39846.11,101.19
3,Full,a1i0p0s1t0,1,0,0.0,1.0,0.0,AR,2022,1,...,182.0,244.39,491.06,-93.09,-60.0,-166.67,17.28,3041.65,36499.75,100.12
4,Full,a1i0p0s0t1,1,0,0.0,0.0,1.0,AR,2022,1,...,182.0,217.02,409.38,-154.49,0.0,-166.67,15.10,2657.41,31888.91,91.70


In [41]:
from sqlalchemy import (create_engine, MetaData, 
                        Table, Column, 
                        Integer, String, 
                        Float, ForeignKey)
from sqlalchemy.orm import declarative_base

In [42]:
engine = create_engine("sqlite:///sss.sqlite", echo = False)
m = MetaData(bind=engine)
Base = declarative_base(metadata=m)

In [43]:
class Miscellaneous(Base):
    __tablename__ = "miscellaneous"
    family_type = Column("family_type", String, primary_key = True)
    place = Column("place", String, primary_key = True)
    state = Column("state", String, primary_key = True)
    year = Column("year", String, primary_key = True)
    analysis =  Column("analysis", String, primary_key = True)
    broadband_and_cell_phone = Column("broadband_and_cell_phone", Float)
    other_necessities = Column("other_necessities", Float)
Base.metadata.create_all(engine)

In [44]:
Session = sessionmaker(bind=engine)
session = Session()

In [45]:
session.bulk_insert_mappings(Miscellaneous,AR2022_clean_col.to_dict(orient="records"))

In [46]:
session.commit()